In [1]:
## File for searching and adding IEX stock data 
## Will search and append all historical data to all stocks 
## Should only be run once, then saved, and not run again in order to conserve API Calls 

## Watch difference between real and sandbox keys for testing purposes 

In [2]:
import pandas as pd

import glob

from pathlib import Path
import csv

import os
import requests
import json

import quandl
from iexfinance.stocks import Stock

from dotenv import load_dotenv
load_dotenv()

True

In [3]:
## Constants 

## Set start date variable - dataframes will be created starting from this date
start_date = '2020-01-01'
end_date = '2021-10-29'
default_date_range = '2y' ## Default Range for IEX functions - don't need more at the moment

In [4]:
## QUANDL/NASDAQ 
nsdq_api_key = os.environ.get('NASDAQ_API_KEY')
base_url_nsdq = 'https://data.nasdaq.com/api/v3/datasets/FINRA/'

iex_api_key = os.getenv("IEX_API_KEY")
iex_test_api_key = os.getenv("IEX_TEST_API_KEY")

## Redundant Assignment but improves Readability throughout code 
real_token = iex_api_key
test_token = iex_test_api_key

base_url_iex = 'https://cloud.iexapis.com/stable/'
sandbox_url = 'https://sandbox.iexapis.com/stable/'

## IEX Status Test 
test_resp = requests.get(base_url_iex + 'status')
test_resp

<Response [200]>

In [5]:
token_status = test_token ## Set to either real token or test token 

In [6]:
## Import FTD File and Symbol List using Pickle 
# ftd_df = load_obj('AnalysisResources/analysis_ftd_all_data.pkl')
# symbol_df = load_obj('AnalysisResources/analysis_symbol_all_list.pkl')


## Import FTD File using CSV 
ftd_df = pd.read_csv(
    Path('../Resources/ftd_all_data.csv'),
    index_col=0, parse_dates=True
)

## Import Symbol and CUSIP list using CSV
symbol_df = pd.read_csv(
    Path('../Resources/symbol_all_list.csv'),
    index_col=0
)

In [7]:
## IEX Calls and Functions 

## Get IEX Chart data - Close Data Only 
def get_iex_chart(stock_ticker, chart_range=default_date_range,token=token_status):
    
    if token == test_token:
        resp_data = requests.get(sandbox_url+'stock/'+stock_ticker+'/chart/'+chart_range+'?chartCloseOnly=true&token='+test_token)
        df = pd.DataFrame(resp_data.json())
    elif token == real_token:
        resp_data = requests.get(base_url_iex+'stock/'+stock_ticker+'/chart/'+chart_range+'?chartCloseOnly=true&token='+real_token)
        df = pd.DataFrame(resp_data.json())

    df.rename(columns={'date':'Date'},inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date',inplace=True)
    
    return df

## Get Short "Interest" Data from Quandl 
def get_short_data_QUANDL(symbol):
    string_nsdq = "FINRA/FNSQ_"+symbol
    string_nyse = "FINRA/FNYX_"+symbol
    
    df1 = quandl.get(string_nsdq,start_date=start_date,end_date=end_date,authtoken=nsdq_api_key)   ## Nasdaq
    df2 = quandl.get(string_nyse,start_date=start_date,end_date=end_date,authtoken=nsdq_api_key)   ## NYSE

    df1 = df1.rename(columns={'ShortVolume':'ShortVolumeNSDQ','TotalVolume':'TotalVolumeNSDQ'})
    #df1 = df1.drop(columns={'ShortExemptVolume'})
    df1 = df1.rename(columns={'ShortExemptVolume':'ShortExemptVolumeNSDQ'})

    df2 = df2.rename(columns={'ShortVolume':'ShortVolumeNYSE','TotalVolume':'TotalVolumeNYSE'})
    #df2 = df2.drop(columns={'ShortExemptVolume'})
    df2 = df2.rename(columns={'ShortExemptVolume':'ShortExemptVolumeNYSE'})

    df3 = pd.merge(df1,df2,on='Date',how='outer')
    
    return df3


## Return FTD Data from SEC FTD files using a Stock's CUSIP number to sort 
def return_ftd_data_cusip(cusip_number):
    df = ftd_df.copy()
    df = df.reset_index()
    df = df.set_index("CUSIP")
    df = df.loc[cusip_number]
    df = df.set_index('Date')
    return df


## Return the CUSIP symbol from the symbol_df symbol list 
def return_CUSIP_from_symbol(symbol):
    new_symbol_df = pd.DataFrame(symbol_df)
    new_symbol_df.dropna(inplace=True)
    new_symbol_df.reset_index(inplace=True,drop=True)
    new_symbol_df.set_index('SYMBOL',inplace=True)
    cusip_variable = new_symbol_df.loc[symbol]
    cusip_variable = cusip_variable['CUSIP']
    return cusip_variable

In [8]:
## Get all time series (iex data and short interest data) and merge with FTD data by passing a single stock symbol 
def get_data_all_time_series(symbol, save=False, token_value=test_token):
    cusip_number = return_CUSIP_from_symbol(symbol)
    ftd_data = return_ftd_data_cusip(cusip_number)
    ftd_data = ftd_data.drop(columns={'SYMBOL'})
    iex_data = get_iex_chart(symbol,token=token_value)
    iex_data = iex_data[start_date:end_date]
    iex_data.reset_index(inplace=True)
    iex_data.rename(columns={'index':'Date'},inplace=True)
    iex_data['Date'] = pd.to_datetime(iex_data['Date'])
    iex_data.set_index('Date',inplace=True)

    ## Merge data
    df1 = pd.merge(iex_data, ftd_data, on='Date',how='outer')
    df1['QUANTITY_FAILS'] = df1['QUANTITY_FAILS'].fillna(0)
    df2 = get_short_data_QUANDL(symbol)
    df = pd.merge(df1,df2,on='Date',how='outer')
    
    ## Export DF based off symbol Name, and parameter
    if save==True:
        pkl_path = Path('FilesExportIndividualSymbol/'+symbol+'_combined_df.pkl')
        save_obj(df, pkl_path)
    
    
    return df 

In [9]:
## Use pickle module to import and export and save files
import pickle
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [10]:
## FMP Functions 
fmpbase_urlv3 = 'https://fmpcloud.io/api/v3/'
fmpbase_urlv4 = 'https://fmpcloud.io/api/v4/'
start_date = '2020-01-01'
end_date = '2021-10-29'
api_key = os.getenv("FMP_CLOUD_API_KEY")
def get_float_data_FMP(symbol):
    url_float_shares = fmpbase_urlv4+'shares_float?symbol='
    url_query_float_data = url_float_shares+symbol+'&apikey='+api_key
    resp_data = requests.get(url_query_float_data)
    #df = pd.DataFrame(resp_data.json())
    json_ = resp_data.json()
    return json_[0]

def get_company_profile_FMP_json(symbol):
    ## https://fmpcloud.io/api/v3/profile/AAPL?apikey='yourkeyhere'
    url_company_profile_url = fmpbase_urlv3+'profile/'+symbol+'?apikey='+api_key
    resp_data = requests.get(url_company_profile_url)
    json_response = resp_data.json()
    return json_response[0]

In [11]:
test_df = get_data_all_time_series('GME',save=False)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 462 entries, 2020-01-02 to 2021-10-29
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   close                  462 non-null    float64
 1   volume                 462 non-null    int64  
 2   change                 462 non-null    float64
 3   changePercent          462 non-null    float64
 4   changeOverTime         462 non-null    float64
 5   QUANTITY_FAILS         462 non-null    float64
 6   ShortVolumeNSDQ        462 non-null    float64
 7   ShortExemptVolumeNSDQ  462 non-null    float64
 8   TotalVolumeNSDQ        462 non-null    float64
 9   ShortVolumeNYSE        462 non-null    float64
 10  ShortExemptVolumeNYSE  462 non-null    float64
 11  TotalVolumeNYSE        462 non-null    float64
dtypes: float64(11), int64(1)
memory usage: 46.9 KB


In [12]:
test_df

,close,volume,change,changePercent,changeOverTime,QUANTITY_FAILS,ShortVolumeNSDQ,ShortExemptVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,ShortExemptVolumeNYSE,TotalVolumeNYSE
Date,,,,,,,,,,,,
2020-01-02,6.35,4528068,0.238432,0.0386,0.048595,10609.0,872127.0,8472.0,1185904.0,276386.0,2448.0,527414.0
2020-01-03,6.08,3583731,-0.447388,-0.0714,-0.025161,28692.0,520156.0,11999.0,832247.0,306258.0,2.0,506189.0
2020-01-06,5.93,3526426,-0.030813,-0.0051,-0.031266,28665.0,498788.0,20569.0,859426.0,198834.0,15.0,319375.0
2020-01-07,5.59,5463729,-0.342297,-0.0580,-0.088384,0.0,604854.0,6372.0,1908324.0,351972.0,991.0,523321.0
2020-01-08,5.86,5767791,0.203119,0.0362,-0.052728,7039.0,746989.0,26168.0,1480915.0,529867.0,28.0,852908.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-25,181.34,1446657,4.306961,0.0258,29.192657,11251.0,224603.0,6814.0,371983.0,55031.0,748.0,114748.0
2021-10-26,179.24,2211288,3.940452,0.0228,29.810502,0.0,338831.0,4819.0,600800.0,75546.0,966.0,167756.0
2021-10-27,175.91,1132600,-4.545253,-0.0247,28.058020,67280.0,164158.0,3233.0,292965.0,34396.0,238.0,84514.0


In [13]:
## Practice making list of dict
## Also a preview of how the upcoming while loop will be setup. 

## Finished variable placeholders 
cusip_number_var = 0
time_series_df = 'dfgoeshere'
FMP_company_profile_json = 0
FMP_float_data_df = 0
market_cap = 0
exchange = 0
   
database_y_test = {'StockSymbol_1':
{
    'Fundamentals':                                             ##
    {
        'sharesOutstanding':'sharesOutstandingValue',          ## Can source for equity but not ETF     
        #'floatShares':'floatSharesValue',                     ## Can't source for ETFs at the moment 
        'marketCap':market_cap,                               ## Sourced FMP
        'debt_ratio':'debt_ratioValue',                        ## Need to source
        'price_earnings':'price_earningsValue',                 ## Need to source
        'exchange':exchange,                                  ## Sourced FMP
        'close_price_value':'last_days_close_price_maybe',        ## Need to source
    },
    'dataFrame':time_series_df,
    'companyProfile':FMP_company_profile_json,
    'floatData':FMP_float_data_df,
    'textNews':['article1','article2','article3'],
    'returns':'returns_data'
}
                  
}

In [14]:
#database_y_test['StockSymbol_1']

In [15]:
database_y_test['StockSymbol_1']['dataFrame']

'dfgoeshere'

In [16]:
## Append database_y_test

data = {'Fundamentals':{
            'sharesOutstanding':'sharesOutstandingValue',
            'floatShares':'floatSharesValue',
            'marketCap':'marketCapValue',
            'debt_ratio':'debt_ratioValue',
            'price_earnings':'price_earningsValue'
        },
        'dataFrame':'dfGoesHere_3',
        'textNews':['article1','article2','article3']  
    }
database_y_test['StockSymbol_2'] = data
database_y_test['StockSymbol_2']['dataFrame']

'dfGoesHere_3'

In [17]:
#fix next thing "make error here"
## Error to stop code from running - need to review next code block before continuing 

In [18]:
## Set iteration through symbol_df 
length_ = len(symbol_df)
loop_token = test_token ## Set to either real_token or test_token 

complete_dict = {}            ## Empty dicts in order to create
incomplete_dict = {}          ## two big files at end of while loop 
stock_dict = {}            ## Dict with completed stock data 
etf_dict = {}              ## Dict with completed ETF data 


x = 0    
y = 50
increment = 50  ## Make sure increment is right
test_length = 50 

## Setup in 50 increments to find error in case the loop fails. 

## If not testing make x <= test_length
while (x < test_length):
    ## Initialize temp dictionaries 
    complete_dict_temp = {}
    incomplete_dict_temp = {}
    ## Initialize temp dictionaries 
    complete_dict_temp_stock = {}
    complete_dict_temp_etf = {}
    
    range_var = range(x,y)
    str_symbol1 = str(x)
    y2 = y - 1 
    str_symbol2 = str(y2)
    
    for i in range_var: 
        ## Iterate through symbol list and create data by symbol
        symbol_var = symbol_df['SYMBOL'][i]
        
        # Use while running loop, if error is made, can see what index it happened on 
        # can also check current value of symbol_var as well 
        error_var = i
        
        ## Get Time Series Data 
        try:
            time_series_df = get_data_all_time_series(symbol_var, token_value=loop_token)
        except KeyError:     ## Some symbols fail and are unreadable. Unreadable symbols are unimportant and okay to be discarded
            continue
        except:
            continue
            
        ## Get FMP Data 
        try: 
            FMP_company_profile_json = get_company_profile_FMP_json(symbol_var)
            if FMP_company_profile_json['isEtf'] == False:
                FMP_float_data_df = get_float_data_FMP(symbol_var)
            else: FMP_float_data_df = 0
            market_cap = FMP_company_profile_json['mktCap']
            exchange = FMP_company_profile_json['exchange']
        except IndexError:
            FMP_company_profile_json = 0
            FMP_float_data_df = 0
            market_cap = 0
            exchange = 0          
        
        
        ## Get Data to add into dict     
        data = {
            'Fundamentals':                                             ##
            {
                'sharesOutstanding':'sharesOutstandingValue',          ## Need to source     
                #'floatShares':'floatSharesValue',                     ## Can't source for ETFs at the moment 
                'marketCap':market_cap,                               ## Sourced FMP
                'debt_ratio':'debt_ratioValue',                        ## Need to source
                'price_earnings':'price_earningsValue',                 ## Need to source
                'exchange':exchange,                                  ## Sourced FMP
                'close_price_value':'last_days_close_price_maybe',        ## Need to source
            },
            'dataFrame':time_series_df,
            'companyProfile':FMP_company_profile_json,
            'floatData':FMP_float_data_df,
            'textNews':['article1','article2','article3'],
            'returns':'returns_data'
        }


            
            
        ## Check if null values, add to different dicts if null values present, or no nulls present
        bool_var = time_series_df.isnull().values.any()
        if bool_var == False:
            complete_dict[symbol_var] = data
            complete_dict_temp[symbol_var] = data
            ## Seperate by Stocks and Equities
            if FMP_company_profile_json != 0:    ## Make sure no error with FMP_company_profile_json
                if FMP_company_profile_json['isEtf'] == False:
                    ## Export stock 
                    stock_dict[symbol_var] = data
                else: ## Export ETF
                    etf_dict[symbol_var] = data  
        elif bool_var == True:
            incomplete_dict[symbol_var] = data
            incomplete_dict_temp[symbol_var] = data

    
    ## Exporting in batches is useful to catch errors, and also pick up where you left off 
    pickle_path1= Path('../FilesExportComplete/data_complete_'+str_symbol1+'_'+str_symbol2+'.pkl')
    save_obj(complete_dict_temp,pickle_path1)
    pickle_path2= Path('../FilesExportIncomplete/data_incomplete_'+str_symbol1+'_'+str_symbol2+'.pkl')
    save_obj(incomplete_dict_temp,pickle_path2)
    
    ## Check before run, if incorrect, can waste a lot of API credits  
    x += increment
    y += increment
    
    ## Export in blocks of {increment} in-case there are errors while processing data. 
    ## Can pick up where the function left off by changing x and y vars to
    ## avoid repeating API calls by doing this in order to not burn 
    ## IEX API tokens unnecessarily 
    
    
## If while loop finishes - export all data 
pkl_path_complete= Path('../Resources/data_complete_all.pkl')
save_obj(complete_dict,pkl_path_complete)
pkl_path_incomplete= Path('../Resources/data_incomplete_all.pkl')
save_obj(incomplete_dict,pkl_path_incomplete)
pkl_path_stocks = Path('../Resources/data_stock_list.pkl')
save_obj(stock_dict,pkl_path_stocks)
pkl_path_etf = Path('../Resources/data_etf_list.pkl')
save_obj(etf_dict,pkl_path_etf)

In [19]:
error_var    #27 


49

In [20]:

symbol_var

'AAMMF'

In [21]:
# json_test = get_company_profile_FMP_json(symbol_var)
# json_test

In [22]:
## Test to see what was exported 
pkl_path_1 = Path('../FilesExportComplete/data_complete_0_49.pkl')
import_data  = load_obj(pkl_path_1)
import_data

## Was successful. Created what was wanted, no errors during process.  
## Can add to and adapt above while loop much further 

{'A': {'Fundamentals': {'sharesOutstanding': 'sharesOutstandingValue',
   'marketCap': 46307536896,
   'debt_ratio': 'debt_ratioValue',
   'price_earnings': 'price_earningsValue',
   'exchange': 'New York Stock Exchange',
   'close_price_value': 'last_days_close_price_maybe'},
  'dataFrame':              close   volume    change  changePercent  changeOverTime  \
  Date                                                                   
  2020-01-02   88.27  1420924  0.655580         0.0078        0.071019   
  2020-01-03   87.72  1122394 -1.404736        -0.0165        0.055038   
  2020-01-06   85.01  2083132  0.260000         0.0030        0.058084   
  2020-01-07   88.55  1765366  0.264903         0.0032        0.062545   
  2020-01-08   87.11  1849073  0.857397         0.0101        0.071445   
  ...            ...      ...       ...            ...             ...   
  2021-10-25  166.18  1130108  0.374369         0.0023        1.019217   
  2021-10-26  159.45  1024012 -0.152818    

In [23]:
key_list1 = []
for key in complete_dict.keys():
    key_list1.append(key)  
len(key_list1)

4

In [24]:
key_list1

['A', 'AA', 'AAAU', 'AAL']

In [25]:
key_list = []
for key in import_data.keys():
    key_list.append(key)  
len(key_list)

4

In [26]:
key_list

['A', 'AA', 'AAAU', 'AAL']

In [27]:
pkl_path_last = Path('../Resources/data_stock_list.pkl')
import_data_last  = load_obj(pkl_path_last)
key_list = []
for key in import_data_last.keys():
    key_list.append(key)  
len(key_list)

3

In [28]:
key_list

['A', 'AA', 'AAL']

In [29]:
import_data_last['A']['floatData']

{'symbol': 'A',
 'date': '2021-11-24',
 'freeFloat': 99.43195331885708,
 'floatShares': 301003400.0,
 'outstandingShares': 302723008.0,
 'source': 'https://www.sec.gov/Archives/edgar/data/1090872/000109087221000015/a-20210731.htm'}